In [ ]:
# -*- coding: utf-8 -*-
import os
import ee
import geemap
Map = geemap.Map
# ======================
# 1) AUTH / CONFIG
# ======================
SERVICE_ACCOUNT = "soilec-service@soiladf2025.iam.gserviceaccount.com"
KEY_FILE        = r"C:\HA\ADF_soilEC_2025\pythoncode\soiladf2025-9f8e370a9642.json"

# Output folder
DOWNLOAD_DIR    = r"C:\HA\zTest\CropmMask"
os.makedirs(DOWNLOAD_DIR, exist_ok=True)

# Assets / collections
GRID_ASSET      = "projects/ee-download-canada/assets/Grid_prairies"

# Export / visualization
EXPORT_SCALE    = 10    # AAFC is 30 m; you can set 10 if you prefer ESA resolution
# ======================
# 2) AUTHENTICATE
# ======================
try:
    creds = ee.ServiceAccountCredentials(SERVICE_ACCOUNT, KEY_FILE)
    ee.Initialize(creds, project="soiladf2025")
except Exception as e:
    raise RuntimeError(f"Failed to initialize EE with service account: {e}")

# ======================
# 3) BUILD MASK FUNCTION
# ======================


# List of classes you want as "1"
TARGET_CLASSES = ee.List([
    132, 133, 134, 135, 136, 137, 138, 139,
    140, 141, 142, 145, 146, 147, 148, 149,
    150, 151, 152, 153, 154, 155, 156, 157,
    158, 160, 162, 167, 174
])

def build_cropland_mask(geom: ee.Geometry) -> ee.Image:
    """
    Builds binary cropland mask: selected AAFC classes → 1, all others → 0.
    """
    # ESA cropland (v100, class 40)
    esa = ee.ImageCollection("ESA/WorldCover/v100").first().clip(geom)
    esa_crop = esa.eq(40)

    # Filter AAFC ACI to time window
    aci = (
        ee.ImageCollection("AAFC/ACI"  )
        .filterDate("2018-01-01", "2024-12-31")
        .map(lambda img: (
            # remap: target classes = 1, default = 0
            img.select(0)
               .remap(TARGET_CLASSES, ee.List.repeat(1, TARGET_CLASSES.size()), 0)
               .rename("cropSel")
               .toUint8()
               .clip(geom)
        ))
    )

    # Collapse multi-year into one mask (any year with target crop = 1)
    aafc_mask = aci.max().rename("aafcMask")

    # Combine with ESA cropland
    mask = (
        ee.Image(aafc_mask)
        .updateMask(esa_crop)
        .rename("mask")
        .unmask(0)
        # .toByte()
        .clip(geom.buffer(30))
    )

    return esa_crop


# # ======================
# # 4) EXAMPLE AOI (SMALL AREA)
# #    Option A: use a small buffer around a point near Saskatoon
# # ======================
# aoi = ee.Geometry.Point([-106.65, 52.15]).buffer(1000).bounds()  # ~5 km buffer, bbox clipped
# mask_img = build_cropland_mask(aoi).clip(aoi)

# # ✅ Use a dedicated map instance (avoid the name 'Map')
# m = geemap.Map(center=[52.15, -106.65], zoom=10)

# # Add layer with visualization (min/max 0/1 for a binary mask)
# m.addLayer(mask_img, {"min": 0, "max": 1, "palette": ["000000", "00FF00"]}, "Cropland mask")
# m.addLayer(aoi, {}, "AOI")

# # In a notebook, just display:
# m


# (Alternatively, to pick one grid cell that intersects this point, uncomment below)
grid = ee.FeatureCollection(GRID_ASSET)
# aoi = grid.filterBounds(ee.Geometry.Point([-106.65, 52.15])).first().geometry()

# ======================
# 5) BUILD MASK & PREVIEW
# ======================
# mask_img = build_cropland_mask(aoi)
# mask_img = ee.Image(mask_img).clip(aoi)
# Map.addLayer(mask_img, {}, "Cropland mask")
# Map

# Quick preview (will open an interactive map in notebook/Colab; in a script it saves an HTML map)
# m = geemap.Map(center=[52.15, -106.65], zoom=10)
# m.addLayer(mask_img, {"min": 0, "max": 1, "palette": MASK_PALETTE}, "Cropland mask")
# m.addLayer(aoi, {}, "AOI")
# preview_html = os.path.join(DOWNLOAD_DIR, "mask_preview.html")
# m.save(preview_html)
# print(f"✓ Preview saved: {preview_html}")

# # ======================
# # 6) DOWNLOAD AS GeoTIFF (LOCAL)
# # ======================
# out_tif = os.path.join(DOWNLOAD_DIR, "cropland_mask_small_aoi.tif")
# geemap.ee_export_image(
#     image=mask_img,
#     filename=out_tif,
#     region=aoi,
#     scale=EXPORT_SCALE,
#     file_per_band=False,
#     crs="EPSG:4326",
#     max_pixels=1e13,
# )
# print(f"✓ GeoTIFF written: {out_tif}")


Map(center=[52.15, -106.65], controls=(WidgetControl(options=['position', 'transparent_bg'], position='toprigh…

In [ ]:
import ee, geemap, os

# ... your auth + build_cropland_mask + AOI code above ...




Map(center=[52.15, -106.65], controls=(WidgetControl(options=['position', 'transparent_bg'], position='toprigh…